In [1]:
# import libraries
from bs4 import BeautifulSoup
import urllib.request #permet d’établir une connexion avec la page web
import csv

In [10]:
# specify the url
urlpage = 'http://www.fasttrack.co.uk/league-tables/tech-track-100/league-table/'
# query the website and return the html to the variable 'page'
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 
request=urllib.request.Request(urlpage,None,headers) #The assembled request
page = urllib.request.urlopen(request)
# parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(page, 'html.parser')
print(soup)

<!-- Template Name: League Table page
-->
<!DOCTYPE html>

<!--[if lt IE 7 ]> <html class="ie ie6 no-js" lang="en-GB"> <![endif]-->
<!--[if IE 7 ]>    <html class="ie ie7 no-js" lang="en-GB"> <![endif]-->
<!--[if IE 8 ]>    <html class="ie ie8 no-js" lang="en-GB"> <![endif]-->
<!--[if IE 9 ]>    <html class="ie ie9 no-js" lang="en-GB"> <![endif]-->
<!--[if gt IE 9]><!-->
<html class="no-js" lang="en-GB">
<!--<![endif]-->
<!-- the "no-js" class is for Modernizr. -->
<head id="live2-fasttrack-com"><link href="https://3s171z3x2dmy48s75v3z1ha8-wpengine.netdna-ssl.com/wp-content/plugins/simple-download-monitor/css/sdm_wp_styles.css?ver=5.6.2" id="sdm-styles-css" media="all" rel="stylesheet" type="text/css"/><link href="https://3s171z3x2dmy48s75v3z1ha8-wpengine.netdna-ssl.com/wp-includes/css/dist/block-library/style.min.css?ver=5.6.2" id="wp-block-library-css" media="all" rel="stylesheet" type="text/css"/>
<meta charset="utf-8"/>
<!-- Always force latest IE rendering engine (even in intranet

In [14]:
# find results within table
table = soup.find('table', attrs={'class': 'tableSorter2'})
results = table.find_all('tr')
print('Number of results', len(results))

Number of results 101


In [15]:
# create and write headers to a list
rows = []
rows.append(['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 'Sales £000s', 'Staff', 'Comments'])
print(rows)

[['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 'Sales £000s', 'Staff', 'Comments']]


In [25]:
# loop over results
for result in results:
    # find all columns per result
    data = result.find_all('td')
    # check that columns have data
    if len(data) == 0:
        continue

    # write columns to variables
    rank = data[0].getText()
    company = data[1].getText()
    location = data[2].getText()
    yearend = data[3].getText()
    salesrise = data[4].getText()
    sales = data[5].getText()
    staff = data[6].getText()
    comments = data[7].getText()
    # Nettoyage des données
    #print('Company is', company)
    #print('Sales', sales)

    # extract description from the name
    companyname = data[1].find('span', attrs={'class':'company-name'}).getText()
    description = company.replace(companyname, '')

    # remove unwanted characters
    sales = sales.strip('*').strip('†').replace(',','')

    # go to link and extract company website
    url = data[1].find('a').get('href')
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 
    request=urllib.request.Request(urlpage,None,headers) #The assembled request
    page = urllib.request.urlopen(request)

    # parse the html
    soup = BeautifulSoup(page, 'html.parser')

    # find the last result in the table and get the link
    try:
        tableRow = soup.find('table').find_all('tr')[-1]
        webpage = tableRow.find('a').get('href')
    except:
        webpage = None

    # write each result to rows
    rows.append([rank, companyname, webpage, description, location, yearend, salesrise, sales, staff, comments])
print(rows)

[['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 'Sales £000s', 'Staff', 'Comments'], ['100', 'Plan.com', 'https://www.fasttrack.co.uk/company_profile/plan-com-3/', 'Communications provider', 'Isle of Man', 'Sep 19', '48.22%', '57260', '91', 'Says it is operating business as usual since March, with customer support, suppliers and services unaffected'], ['1', 'Revolut', 'https://www.fasttrack.co.uk/company_profile/plan-com-3/', 'Digital banking services provider', 'East London', 'Dec 19', '309.89%', '162722', '1,289', 'Launched in America on the same day that the UK locked down'], ['2', 'Football Index', 'https://www.fasttrack.co.uk/company_profile/plan-com-3/', 'Football trading platform', 'Central London', 'Dec 19', '306.02%', '45120', '64', 'Paid out double cash dividends to its traders during lockdown'], ['3', 'Tessian', 'https://www.fasttrack.co.uk/company_profile/plan-com-3/', 'Email security provider', 'Central London',

In [26]:
# Create csv and write rows to output file
with open('techtrack100.csv','w', newline='') as f_output:
    csv_output = csv.writer(f_output)
    csv_output.writerows(rows)